# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [61]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/rsoares/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rsoares/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [62]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [63]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [64]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [66]:
print(f"Number of documents (PDF pages): {len(docs)}\n")
for i, doc in enumerate(docs[:2]):
    print(f"Document {i+1} page_content:\n{doc.page_content}\n")

Number of documents (PDF pages): 64

Document 1 page_content:
NBER WORKING PAPER SERIES
HOW PEOPLE USE CHATGPT
Aaron Chatterji
Thomas Cunningham
David J. Deming
Zoe Hitzig
Christopher Ong
Carl Yan Shan
Kevin Wadman
Working Paper 34255
http://www.nber.org/papers/w34255
NATIONAL BUREAU OF ECONOMIC RESEARCH
1050 Massachusetts Avenue
Cambridge, MA 02138
September 2025
We acknowledge help and comments from Joshua Achiam, Hemanth Asirvatham, Ryan 
Beiermeister, Rachel Brown, Cassandra Duchan Solis, Jason Kwon, Elliott Mokski, Kevin Rao, 
Harrison Satcher, Gawesha Weeratunga, Hannah Wong, and Analytics & Insights team. We 
especially thank Tyna Eloundou and Pamela Mishkin who in several ways laid the foundation for 
this work. This study was approved by Harvard IRB (IRB25-0983). A repository containing all 
code run to produce the analyses in this paper is available on request. The views expressed herein 
are those of the authors and do not necessarily reflect the views of the National Bureau

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [67]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

![RAGAS Knowledge Graph diagram](https://docs.ragas.io/en/stable/_static/imgs/kg_rag.png)

In [68]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [69]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

**It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.**

In [70]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '5d6be7'. Skipping!
Property 'summary' already exists in node 'b1cfc3'. Skipping!
Property 'summary' already exists in node 'a7443a'. Skipping!
Property 'summary' already exists in node 'b42e24'. Skipping!
Property 'summary' already exists in node '5ead91'. Skipping!
Property 'summary' already exists in node '237242'. Skipping!
Property 'summary' already exists in node 'd7812e'. Skipping!
Property 'summary' already exists in node '2808eb'. Skipping!
Property 'summary' already exists in node '99e090'. Skipping!
Property 'summary' already exists in node '983599'. Skipping!
Property 'summary' already exists in node '7f645e'. Skipping!
Property 'summary' already exists in node 'ba9ff9'. Skipping!
Property 'summary' already exists in node '4b116f'. Skipping!
Property 'summary' already exists in node 'e202a0'. Skipping!
Property 'summary' already exists in node '5073b2'. Skipping!
Property 'summary' already exists in node 'd5c8d0'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'ba9ff9'. Skipping!
Property 'summary_embedding' already exists in node '237242'. Skipping!
Property 'summary_embedding' already exists in node 'a7443a'. Skipping!
Property 'summary_embedding' already exists in node '99e090'. Skipping!
Property 'summary_embedding' already exists in node '2808eb'. Skipping!
Property 'summary_embedding' already exists in node 'b42e24'. Skipping!
Property 'summary_embedding' already exists in node '5ead91'. Skipping!
Property 'summary_embedding' already exists in node 'd7812e'. Skipping!
Property 'summary_embedding' already exists in node '7f645e'. Skipping!
Property 'summary_embedding' already exists in node '5d6be7'. Skipping!
Property 'summary_embedding' already exists in node 'b1cfc3'. Skipping!
Property 'summary_embedding' already exists in node '983599'. Skipping!
Property 'summary_embedding' already exists in node '4b116f'. Skipping!
Property 'summary_embedding' already exists in node 'e202a0'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 712)

We can save and load our knowledge graphs as follows.

In [71]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 712)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [72]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

**In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data** - and then use a scenario to construct a question from that data and persona.

In [73]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answear:
They represent diffenrent types of questions a user can use while interactiong with a RAG system.

 * **SingleHop** queries represent a fact-based question that can be answered with a single retrieval from a document source.
 * **MultiHop** queries involves multiple steps of reasoning, requiring information from two or more sources. 
 * **Specific** queries focuses on clear, fact-based retrieval. The goal in RAG is to retrieve highly relevant information from one or more documents that directly address the specific question.
 * **Abstract** queries requires a broader, more interpretive response. In RAG, abstract queries challenge the retrieval system to pull from documents that contain higher-level reasoning, explanations, or opinions, rather than simple facts.

Finally, we can use our `TestSetGenerator` to generate our testset!

The diagram below is a representation of a RAGAS Testset:

![](https://docs.ragas.io/en/stable/_static/imgs/scenario_rag.png)

In [74]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What does July 2025 mean for ChatGPT users?,[Introduction ChatGPT launched in November 202...,"By July 2025, 18 billion messages were being s...",single_hop_specifc_query_synthesizer
1,What is Handa et al. (2025) about in relation ...,[Table 1: ChatGPT daily message counts (millio...,Handa et al. (2025) report on ChatGPT daily me...,single_hop_specifc_query_synthesizer
2,Section what does it mean in the context of Ch...,[Variation by Occupation Figure 23 presents va...,The context discusses variation in ChatGPT usa...,single_hop_specifc_query_synthesizer
3,July 2025 use ChatGPT how many,[Conclusion This paper studies the rapid growt...,"By July 2025, ChatGPT had been used weekly by ...",single_hop_specifc_query_synthesizer
4,how chatgpt use differ by occupation and priva...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context shows that ChatGPT usage varies si...,multi_hop_abstract_query_synthesizer
5,ChatGPT use vs other AI like Claude impact wor...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context shows ChatGPT mostly used for prac...,multi_hop_abstract_query_synthesizer
6,How does the regression analysis of occupation...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,"The regression analysis of occupation effects,...",multi_hop_abstract_query_synthesizer
7,Based on the rapid growth of ChatGPT usage in ...,[<1-hop>\n\nConclusion This paper studies the ...,"The rapid growth of ChatGPT in the US, with mo...",multi_hop_specific_query_synthesizer
8,How has the usage of ChatGPT evolved by July 2...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT experienced rapid growth...",multi_hop_specific_query_synthesizer
9,"Hwo do Handa et al., 2025, describe the differ...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,"Handa et al., 2025, report that while most Cha...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'bdb275'. Skipping!
Property 'summary' already exists in node '70ec49'. Skipping!
Property 'summary' already exists in node 'a12b37'. Skipping!
Property 'summary' already exists in node '3e580d'. Skipping!
Property 'summary' already exists in node '5012b3'. Skipping!
Property 'summary' already exists in node '09b909'. Skipping!
Property 'summary' already exists in node '73c399'. Skipping!
Property 'summary' already exists in node '4c6efd'. Skipping!
Property 'summary' already exists in node 'fe9738'. Skipping!
Property 'summary' already exists in node 'f1ed55'. Skipping!
Property 'summary' already exists in node '1262c2'. Skipping!
Property 'summary' already exists in node '34e7d1'. Skipping!
Property 'summary' already exists in node '2c169a'. Skipping!
Property 'summary' already exists in node '7b5c6f'. Skipping!
Property 'summary' already exists in node '0471b2'. Skipping!
Property 'summary' already exists in node '74895b'. Skipping!
Property

...

Property 'summary_embedding' already exists in node 'fe9738'. Skipping!
Property 'summary_embedding' already exists in node '4c6efd'. Skipping!
Property 'summary_embedding' already exists in node 'bdb275'. Skipping!
Property 'summary_embedding' already exists in node '73c399'. Skipping!
Property 'summary_embedding' already exists in node '09b909'. Skipping!
Property 'summary_embedding' already exists in node 'a12b37'. Skipping!
Property 'summary_embedding' already exists in node '1262c2'. Skipping!
Property 'summary_embedding' already exists in node '5012b3'. Skipping!
Property 'summary_embedding' already exists in node 'f1ed55'. Skipping!
Property 'summary_embedding' already exists in node '3e580d'. Skipping!
Property 'summary_embedding' already exists in node '70ec49'. Skipping!
Property 'summary_embedding' already exists in node '34e7d1'. Skipping!
Property 'summary_embedding' already exists in node '0471b2'. Skipping!
Property 'summary_embedding' already exists in node '78472e'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
dataset.to_pandas().to_csv("synthetic_testset.csv", index=False)
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"How significant is the usage of ChatGPT, with ...",[Introduction ChatGPT launched in November 202...,"By July 2025, 18 billion messages were being s...",single_hop_specifc_query_synthesizer
1,What is Claude in the context of ChatGPT usage?,[Table 1: ChatGPT daily message counts (millio...,Claude is mentioned as a term in the context o...,single_hop_specifc_query_synthesizer
2,ChatGPT use in jobs?,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 shows variat...,single_hop_specifc_query_synthesizer
3,What does Seeking Information mean in ChatGPT ...,[Conclusion This paper studies the rapid growt...,Seeking Information is one of the three most c...,single_hop_specifc_query_synthesizer
4,Jun 2024 message volume vs Jun 2025 non-work m...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In June 2024, there were 451 million total mes...",multi_hop_abstract_query_synthesizer
5,How does ChatGPT usage vary across occupation ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,ChatGPT usage varies significantly across occu...,multi_hop_abstract_query_synthesizer
6,How does the rapid growth and widespread adopt...,[<1-hop>\n\nConclusion This paper studies the ...,"The rapid growth of ChatGPT, with over 700 mil...",multi_hop_abstract_query_synthesizer
7,"In June 2024 and June 2025, how did the growth...",[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"According to the data, in June 2024, non-work ...",multi_hop_abstract_query_synthesizer
8,Considering the rapid growth of ChatGPT usage ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, approximately 70% of ChatGPT con...",multi_hop_specific_query_synthesizer
9,How does the usage of ChatGPT in July 2025 ref...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT had been used weekly by ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [ ]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [ ]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [ ]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [ ]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

"Based on the provided context, people are using AI, particularly generative AI like ChatGPT, for a variety of activities both at work and outside of work. These uses include performing workplace tasks by either augmenting or automating human labor, producing writing, software code, spreadsheets, and other digital products. People also use AI to seek information and advice, improve productivity, solve problems with AI as co-workers or co-pilots, and engage in self-expression such as relationships, personal reflection, games, and role play. Generative AI's flexibility allows it to be applied in many different ways across diverse tasks."

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [ ]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

##### ✅ Answer:

- `qa_evaluator`: correctness of the LLM response to a user query or question
- `labeled_helpfulness_evaluator`: helpsulness of the LLM response
- `dopeness_evaluator`: how "dope" (lit, cool) the LLM response is

## LangSmith Evaluation

In [ ]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'weary-idea-78' at:
https://smith.langchain.com/o/bb9a3dd7-ec88-4998-a3cb-7a19da336bd6/datasets/c860f007-b15f-4ab5-b7a3-f878ec0967f2/compare?selectedSessions=8a19434f-539f-4cd0-bdf1-5664f7106f13




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,when november 2022 chatgpt start and how many ...,ChatGPT was released to the public on November...,None,"ChatGPT launched in November 2022, and by July...",1,1,0,2.802712,252cd427-19a1-4caa-9750-8a69d12ca792,28cefe55-2c28-4960-b570-8e2acb60a366
1,how many users 700 million users use ChatGPT f...,"Based on the context, the 700 million weekly a...",None,"By July 2025, ChatGPT was used weekly by more ...",1,0,0,4.708007,9ba91298-9a7c-4614-b16f-ac918bfcee96,f5d48f14-66d4-46f7-9c68-68aa00eaf824
2,How does the usage of ChatGPT in July 2025 ref...,"By July 2025, ChatGPT had been adopted by arou...",None,"By July 2025, ChatGPT had been used weekly by ...",1,1,0,2.434548,e28a49ee-4b25-4600-857b-5cb89a2691a8,2163b65c-53dd-4f23-a002-4c89b21ea585
3,Considering the rapid growth of ChatGPT usage ...,"By July 2025, non-work-related ChatGPT message...",None,"By July 2025, approximately 70% of ChatGPT con...",1,1,0,3.002660,b892e263-476b-4e1b-870f-0efefa869575,aaaa9000-a13a-449e-a300-596b11d60346
4,"In June 2024 and June 2025, how did the growth...",Based on the provided context:\n\n- The three ...,None,"According to the data, in June 2024, non-work ...",1,1,0,9.531043,8d6eec9b-1079-4d86-b3ad-feb17755d1d1,211f4add-c9de-4536-9819-afec3b696566
5,How does the rapid growth and widespread adopt...,The rapid growth and widespread adoption of Ch...,None,"The rapid growth of ChatGPT, with over 700 mil...",1,1,0,2.744580,7c6da625-f27e-4823-bd7f-b8eb559b69ef,42766ec3-5293-4b1c-ac03-4b0b82c21fa9
6,How does ChatGPT usage vary across occupation ...,ChatGPT usage varies across broad occupation c...,None,ChatGPT usage varies significantly across occu...,1,1,0,3.730170,b17c8070-a3d3-4848-9d38-b5c209e25443,3e472bf8-a8d2-4087-ba68-b655ca8d9ad8
7,Jun 2024 message volume vs Jun 2025 non-work m...,"In June 2024, there were 238 million non-work ...",None,"In June 2024, there were 451 million total mes...",1,1,0,2.685718,2aaf188f-3b37-47ed-984d-dd9f8682a040,0f51f277-f99e-40be-b0ff-2c5084798920
8,What does Seeking Information mean in ChatGPT ...,Seeking Information in ChatGPT usage refers to...,None,Seeking Information is one of the three most c...,0,1,0,2.283763,526e5eba-10be-4ceb-b4bc-0c710f0ade09,fb78ecb0-2d26-4d55-8abb-6ad90d0b6a76
9,ChatGPT use in jobs?,ChatGPT usage at work is more common among use...,None,Variation by Occupation Figure 23 shows variat...,1,1,0,3.773399,729153ad-5076-46b7-a685-0a14cfdd4bae,0bf22845-c8f6-445c-895a-53b3d91e0d8d


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [76]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [77]:
rag_documents = docs

In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:
With bigger chuncks our embeddigns can capture more meaning as each chunck contain more context/information. But it is important to figure it out an optimal size for the type of document in each use case scenario.

In [79]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [80]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [81]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [82]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [83]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, let’s crank that insight dial to 11 and dive into this AI-money-making magic based purely on the rad context you dropped!\n\nPeople aren’t just using AI like ChatGPT to push buttons and get basic tasks done—they’re flexing it as a **research assistant and decision-support co-pilot** in the workplace. This isn’t about AI replacing jobs but supercharging *how* humans make decisions and crank up productivity, especially in brain-heavy, knowledge-intensive gigs. \n\nThat means AI’s beast mode is in:\n- **Advising and brainstorming** to make smarter work moves.\n- **Boosting output** by giving users sharper info and guidance so they make killer decisions that pay dividends.\n- And since knowledge work is a cash cow driven by decision quality, AI basically levels up your economic hustle by juicing your cognitive horsepower.\n\nPlus, the economics nerds Collis and Brynjolfsson (2025) put some wild numbers on this vibe — folks would shell out nearly a hundred bucks a month just to ke

Finally, we can evaluate the new chain on the same test set!

In [84]:
dataset_name

'Use Case Synthetic Data - AIE8'

In [85]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'damp-shade-24' at:
https://smith.langchain.com/o/bb9a3dd7-ec88-4998-a3cb-7a19da336bd6/datasets/c860f007-b15f-4ab5-b7a3-f878ec0967f2/compare?selectedSessions=3775cda9-63fa-4cd8-9efd-184d7304fb6e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,when november 2022 chatgpt start and how many ...,"Oh yeah, diving into the chronicles of ChatGPT...",None,"ChatGPT launched in November 2022, and by July...",1,1,1,4.596144,252cd427-19a1-4caa-9750-8a69d12ca792,fca7cf54-7a4b-45eb-adb5-a0134dea0950
1,how many users 700 million users use ChatGPT f...,"Alright, let’s drop some straight-up knowledge...",None,"By July 2025, ChatGPT was used weekly by more ...",1,0,1,3.890149,9ba91298-9a7c-4614-b16f-ac918bfcee96,cbdfa4af-6a83-4a83-babb-ceb76c7a8e8c
2,How does the usage of ChatGPT in July 2025 ref...,"Oh, buckle up for this stellar ride through Ch...",None,"By July 2025, ChatGPT had been used weekly by ...",1,1,1,6.613110,e28a49ee-4b25-4600-857b-5cb89a2691a8,957915bf-7264-43f5-ada1-0bd4c360c850
3,Considering the rapid growth of ChatGPT usage ...,"Yo, by July 2025, non-work-related ChatGPT mes...",None,"By July 2025, approximately 70% of ChatGPT con...",1,1,1,3.935129,b892e263-476b-4e1b-870f-0efefa869575,e65adb6d-f49e-4b4c-a167-5e244ac99e2b
4,"In June 2024 and June 2025, how did the growth...","Yo, let’s break down this ChatGPT saga like a ...",None,"According to the data, in June 2024, non-work ...",1,1,1,7.712247,8d6eec9b-1079-4d86-b3ad-feb17755d1d1,4a26a9a0-7879-4ea8-bfe7-d1993d876a5f
5,How does the rapid growth and widespread adopt...,"Alright, buckle up—here’s the rad scoop straig...",None,"The rapid growth of ChatGPT, with over 700 mil...",1,1,1,5.253360,7c6da625-f27e-4823-bd7f-b8eb559b69ef,a3dc019d-1284-4a55-983e-00616ddb0d71
6,How does ChatGPT usage vary across occupation ...,"Alright, buckle up — here’s the lowdown on Cha...",None,ChatGPT usage varies significantly across occu...,1,1,1,8.785578,b17c8070-a3d3-4848-9d38-b5c209e25443,729a92b1-b7bf-4ba0-87ac-1586e84eee6b
7,Jun 2024 message volume vs Jun 2025 non-work m...,"Alright, let’s dive into this epic ChatGPT mes...",None,"In June 2024, there were 451 million total mes...",1,1,1,6.617192,2aaf188f-3b37-47ed-984d-dd9f8682a040,390de5ec-2b0d-4c6e-92d1-8edc5e7da039
8,What does Seeking Information mean in ChatGPT ...,"Alright, buckle up for some top-tier AI knowle...",None,Seeking Information is one of the three most c...,0,0,1,3.777026,526e5eba-10be-4ceb-b4bc-0c710f0ade09,9d18202c-44e1-4519-b8c4-c43fdc6a7772
9,ChatGPT use in jobs?,"Alright, buckle up because here’s the rad lowd...",None,Variation by Occupation Figure 23 shows variat...,1,1,1,4.447190,729153ad-5076-46b7-a685-0a14cfdd4bae,a1648c31-86bd-4762-a18d-9f665972f4b1


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:
The stats below show the difference between two evaluation runs shwoing a hugge difference in the "dopeness" criteria evaluator. 

![](./langsmith-evals-comparison.png)

The first experiment run (labeled as "**weary-idea-78**") shows the dopeness evaluation with zeroed score for all queries.

![](./langsmith-bad-dopeness-eval.png)

After applying some changes in th RAG chain, like: 
 * using bigger chuncks and a bigger embedding models; 
 * rewording the prompt;

we can see in the second experiment (labeled as "**damp-shade-24**") a significant change on how the dopeness evaluator gets scored.

![](./langsmith-good-dopness-eval.png)

Based on the results from the second experiment we can infer that the changes applied to the RAG chain helped the model better understand the context from the "dopeness perspective".  